In [34]:
import numpy as np
from scipy.optimize import minimize

In [35]:
x = [0, 0, 0, 1, 1, 1]
y = np.array([[-1, -1], [-1, -2], [-2, -1], [1, 1], [1, 2], [2, 1]])
M = len(y[0])
n = len(x)
phi = [np.array([1, 0] if xx == 0 else [0, 1]) for xx in x]

In [40]:
def fun(W):
    W.shape = (2, 2)
    total = 0
    for j in range(M):
        for i in range(n):
            total += (y[i, j] - W[:, j].dot(phi[i])) ** 2
    return total
            
minimize(fun, 2 * np.ones(4))

   status: 0
  success: True
     njev: 3
     nfev: 12
 hess_inv: array([[1, 0],
       [0, 1]])
      fun: 24.0
        x: array([[  8.88178420e-16,   8.88178420e-16],
       [  8.88178420e-16,   8.88178420e-16]])
  message: 'Optimization terminated successfully.'
      jac: array([  0.00000000e+00,   4.76837158e-07])